In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def generate_data(num_samples=1000):
    np.random.seed(42)
    x = np.random.uniform(-2, 2, num_samples)
    y = np.random.uniform(-2, 2, num_samples)
    z = np.random.uniform(-2, 2, num_samples)
    f_xyz = np.sin(x) + np.cos(y) * np.power(z, 2)
    features = np.vstack((x, y, z)).T
    outputs = f_xyz
    return features, outputs

features, outputs = generate_data()  # Generates synthetic dataset


# **Low-Level API**

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

class CustomNNLowLevel(tf.Module):
    def __init__(self, input_size=3, hidden_size=64, output_size=1):
        super().__init__()
        # Initialize weights and biases using low-level TensorFlow operations
        self.W1 = tf.Variable(tf.random.truncated_normal([input_size, hidden_size], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([hidden_size]))
        self.W2 = tf.Variable(tf.random.truncated_normal([hidden_size, hidden_size], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([hidden_size]))
        self.W3 = tf.Variable(tf.random.truncated_normal([hidden_size, output_size], stddev=0.1))
        self.b3 = tf.Variable(tf.zeros([output_size]))

    def __call__(self, x):
        x = tf.add(tf.matmul(x, self.W1), self.b1)
        x = tf.nn.relu(x)
        x = tf.add(tf.matmul(x, self.W2), self.b2)
        x = tf.nn.relu(x)
        x = tf.add(tf.matmul(x, self.W3), self.b3)
        return x

# Assume generate_data() is defined as shown previously
features, outputs = generate_data(1000) # Generate synthetic data
X_train, X_test, y_train, y_test = train_test_split(features, outputs, test_size=0.2, random_state=42)

# Training process similar to previous examples, using CustomNNLowLevel and manual gradients



In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.01)

def train_step(model, inputs, outputs, optimizer):
    with tf.GradientTape() as tape:
        current_loss = tf.reduce_mean(tf.square(model(inputs) - outputs))
    gradients = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return current_loss



In [ ]:
def train(model, X_train, y_train, optimizer, epochs=100):
    for epoch in range(epochs):
        loss = train_step(model, X_train, y_train, optimizer)
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Loss: {loss.numpy()}")

# Convert data to tensors
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)

# Initialize the model and start training
model = CustomNNLowLevel()
train(model, X_train_tf, y_train_tf, optimizer)


Epoch 0: Loss: 2.011073350906372
Epoch 10: Loss: 1.7068597078323364
Epoch 20: Loss: 1.6846498250961304
Epoch 30: Loss: 1.6806187629699707
Epoch 40: Loss: 1.6788954734802246
Epoch 50: Loss: 1.6771711111068726
Epoch 60: Loss: 1.6765714883804321
Epoch 70: Loss: 1.6763465404510498
Epoch 80: Loss: 1.6762698888778687
Epoch 90: Loss: 1.6762449741363525


In [ ]:
def evaluate(model, X_test, y_test):
    X_test_tf = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)
    predictions = model(X_test_tf)
    mse_loss = tf.reduce_mean(tf.square(predictions - y_test_tf))
    print(f"Test MSE: {mse_loss.numpy()}")

evaluate(model, X_test, y_test)


Test MSE: 1.6408116817474365
